In [1]:
import pandas as pd
import requests
import time
import threading


url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
payload = {
    "grant_type": "password",
    "client_id": "351fb11916fc85504ba5f6ec3b96988b",
    "client_secret": "8f9355878504005ba782ca049d013d4e",
    "redirect_uri": "https://crm-p10.xiaoshouyi.com",
    "username": "11020964@twkd.com",
    "password": "Kd0627ubGHagpQ"
}


response = requests.post(url, data=payload)
content = response.json()
ac_token = content["access_token"]

'''
complete K on new crm 提交9/1後面的資料
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
TW_OS_account = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''SELECT id, customItem253__c, accountCode__c, createdAt, approvalStatus, customItem223__c as 附件,
                         customItem292__c as 資料來源
                   FROM account
                   where createdAt >='1725148800000' and approvalStatus = 'Unsubmitted'
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    TW_OS_account = pd.concat([TW_OS_account, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
TW_OS_account['資料來源'] = TW_OS_account['資料來源'].str.get(0)
tw_df = TW_OS_account[TW_OS_account['customItem253__c'].str.startswith("TW")]
os_df = TW_OS_account[~TW_OS_account['customItem253__c'].str.startswith("TW")]
tw_df = tw_df[~tw_df['附件'].isna()]
os_df = os_df[~os_df['附件'].isna()]

In [ ]:

status_url = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/creekflow/task/actions/preProcessor"  #獲取下一步訊息
task_url = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/creekflow/task"                         #審批
headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type": "application/json"
    }

def preProcessor(process_ID):
    status_body = {
        "data": {
            "action": "submit",
            "entityApiKey": "account",
            "dataId": process_ID }}

    response = requests.post(status_url, headers=headers, json=status_body)
    crm_json = response.json()['data']
    return crm_json

if not tw_df.empty:
    tw_df_last_row = tw_df.iloc[-1]
    tw_approval_status = preProcessor(tw_df_last_row['id'])
    next_Approver = str([approver.get('id') for approver in tw_approval_status['chooseApprover'] if 'id' in approver][0])
    tw_df['next_Approver'] = next_Approver
    
    def submit_task(row):
        data_id = row['id']
        task_id = row['next_Approver']
        url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/creekflow/task"
        headers = {
            "Authorization": f"Bearer {ac_token}",
            "Content-Type": "application/json"
        }

        # Updated nextAssignees/ccs list

        data = {
            "data": {
                "action": "submit",
                "entityApiKey": "account",
                "dataId": data_id,
                "procdefId": tw_approval_status['procdefId'],
                "nextTaskDefKey": tw_approval_status['nextTaskDefKey'],
                "nextAssignees": [task_id],
                "ccs": [task_id]  
            }
        }

        response = requests.post(url_2, headers=headers, json=data)
        result = response.json()
        print(f"Response for dataId {data_id}: {result}")

    # Assuming data_ids is your DataFrame
    data_ids_df = tw_df[['id', 'next_Approver']]

    # Create a thread for each row in the DataFrame

    threads = []
    for index, row in data_ids_df.iterrows():
        thread = threading.Thread(target=submit_task, args=(row,))
        threads.append(thread)
        thread.start()
        time.sleep(0.08)

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

if not os_df.empty:
    os_df_last_row = os_df.iloc[-1]
    os_approval_status = preProcessor(os_df_last_row['id'])
    os_next_Approver = str([approver.get('id') for approver in os_approval_status['chooseApprover'] if 'id' in approver][0])
    os_df['next_Approver'] = os_next_Approver



    def submit_task_os(row):
        data_id = row['id']
        task_id = row['next_Approver']
        url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/creekflow/task"
        headers = {
            "Authorization": f"Bearer {ac_token}",
            "Content-Type": "application/json"
        }
    
        # Updated nextAssignees/ccs list
    
        data = {
            "data": {
                "action": "submit",
                "entityApiKey": "account",
                "dataId": data_id,
                "procdefId": os_approval_status['procdefId'],
                "nextTaskDefKey": os_approval_status['nextTaskDefKey'],
                "nextAssignees": [task_id],
                "ccs": [task_id]  
            }
        }
    
        response = requests.post(url_2, headers=headers, json=data)
        result = response.json()
        print(f"Response for dataId {data_id}: {result}")
    
    # Assuming data_ids is your DataFrame
    data_ids_df = os_df[['id', 'next_Approver']]
    
    # Create a thread for each row in the DataFrame
    
    threads = []
    for index, row in data_ids_df.iterrows():
        thread = threading.Thread(target=submit_task_os, args=(row,))
        threads.append(thread)
        thread.start()
        time.sleep(0.08)
    
    # Wait for all threads to finish
    for thread in threads:
        thread.join()

Response for dataId 3728784844885997: {'code': '1602066', 'msg': '参数"下一步"的值不正确，请重新提交', 'data': None, 'errorInfo': None}
Response for dataId 3727319075835847: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3727337391577427: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3728686072764750: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3728982611057002: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
